# Load Questions and Question Sources to Database

This notebook loads:
1. `3_question_gen.csv` into the `faq_questions` table
2. `3_question_source_gen.csv` into the `faq_question_sources` table

In [12]:
import pandas as pd
from pathlib import Path
from database.backends.factory import BackendFactory
from database.config import DatabaseConfig
from database.repository import FAQRepository

In [13]:
# Define file paths
questions_csv = Path('data_ingestion/data/3_question_gen.csv')
question_sources_csv = Path('data_ingestion/data/3_question_source_gen.csv')
db_path = Path('databases/faq_update.db')

# Create database configuration
config = DatabaseConfig(backend='sqlite', db_path=str(db_path))
print(f"Database: {db_path}")

Database: databases\faq_update.db


In [14]:
# Load CSV files
print("Loading questions CSV...")
df_questions = pd.read_csv(questions_csv)
print(f"Loaded {len(df_questions)} questions")
print("\nQuestions columns:", df_questions.columns.tolist())
print("\nFirst few rows:")
df_questions.head()

Loading questions CSV...
Loaded 74 questions

Questions columns: ['question_id', 'question_text', 'source_type', 'generation_method', 'status', 'created_at', 'modified_at']

First few rows:


question_id  \
0  f29aefee41298f9eddc31cbbc10e3970dc067f22143c7f...   
1  b753cc4091103cd521842592a7ee66188af23b39a48596...   
2  4641cee4e692f548be4f9de631211fdd1903ea1a23060c...   
3  8d64db97b362e163b244cf39b6314ecb09a7525b11b0df...   
4  7b8d4cc8962467ca328c7cfd748d24322b4414238875f9...   

                                       question_text source_type  \
0  When does the health insurance enrollment peri...    document   
1  What are the health insurance plan options ava...    document   
2  When does health insurance coverage become eff...    document   
3  Can dependents be added to health insurance pl...    document   
4  Are pre-existing conditions covered under the ...    document   

  generation_method  status                created_at  \
0               LLM  active  2025-10-29T19:31:55.254Z   
1               LLM  active  2025-10-29T19:31:55.254Z   
2               LLM  active  2025-10-29T19:31:55.254Z   
3               LLM  active  2025-10-29T19:31:55.254Z   
4               LLM  active  2025-10-29T19:31:55.254Z   

                modified_at  
0  2025-10-29T19:31:55.255Z  
1  2025-10-29T19:31:55.255Z  
2  2025-10-29T19:31:55.255Z  
3  2025-10-29T19:31:55.255Z  
4  2025-10-29T19:31:55.255Z

In [15]:
print("Loading question sources CSV...")
df_question_sources = pd.read_csv(question_sources_csv)
print(f"Loaded {len(df_question_sources)} question sources")
print("\nQuestion sources columns:", df_question_sources.columns.tolist())
print("\nFirst few rows:")
df_question_sources.head()

Loading question sources CSV...
Loaded 74 question sources

Question sources columns: ['source_id', 'question_id', 'content_checksum', 'is_primary_source', 'contribution_weight', 'is_valid', 'valid_from', 'valid_until', 'invalidation_reason', 'invalidated_by_change_id', 'created_at']

First few rows:


source_id                                        question_id  \
0          1  f29aefee41298f9eddc31cbbc10e3970dc067f22143c7f...   
1          2  b753cc4091103cd521842592a7ee66188af23b39a48596...   
2          3  4641cee4e692f548be4f9de631211fdd1903ea1a23060c...   
3          4  8d64db97b362e163b244cf39b6314ecb09a7525b11b0df...   
4          5  7b8d4cc8962467ca328c7cfd748d24322b4414238875f9...   

                                    content_checksum  is_primary_source  \
0  370cd156d5ae42b3bb93fe51096b05d8dc2aee7358aaee...               True   
1  370cd156d5ae42b3bb93fe51096b05d8dc2aee7358aaee...               True   
2  370cd156d5ae42b3bb93fe51096b05d8dc2aee7358aaee...               True   
3  370cd156d5ae42b3bb93fe51096b05d8dc2aee7358aaee...               True   
4  370cd156d5ae42b3bb93fe51096b05d8dc2aee7358aaee...               True   

   contribution_weight  is_valid                valid_from  valid_until  \
0                    1      True  2025-10-29T19:31:59.431Z          NaN   
1                    1      True  2025-10-29T19:31:59.431Z          NaN   
2                    1      True  2025-10-29T19:31:59.431Z          NaN   
3                    1      True  2025-10-29T19:31:59.431Z          NaN   
4                    1      True  2025-10-29T19:31:59.431Z          NaN   

   invalidation_reason  invalidated_by_change_id                created_at  
0                  NaN                       NaN  2025-10-29T19:31:59.432Z  
1                  NaN                       NaN  2025-10-29T19:31:59.432Z  
2                  NaN                       NaN  2025-10-29T19:31:59.432Z  
3                  NaN                       NaN  2025-10-29T19:31:59.432Z  
4                  NaN                       NaN  2025-10-29T19:31:59.432Z

## Data Transformation

Transform CSV values to match database enum values:
- `source_type`: "document" → "from_documents"
- `generation_method`: "LLM" → "llm_generated"

In [16]:
# Transform enum values to match database models
print("🔄 Transforming data to match database enum values...")

# Mapping for source_type
source_type_mapping = {
    'document': 'from_documents',
    'user_query': 'from_user_queries', 
    'manual': 'from_manual',
    'validation': 'from_validation'
}

# Mapping for generation_method
generation_method_mapping = {
    'LLM': 'llm_generated',
    'manual': 'human_written',
    'extracted': 'extracted'
}

# Apply transformations
if 'source_type' in df_questions.columns:
    df_questions['source_type'] = df_questions['source_type'].map(source_type_mapping)
    print(f"✓ Transformed source_type: {df_questions['source_type'].unique()}")

if 'generation_method' in df_questions.columns:
    df_questions['generation_method'] = df_questions['generation_method'].map(generation_method_mapping)
    print(f"✓ Transformed generation_method: {df_questions['generation_method'].unique()}")

print()
print("📋 Transformed data sample:")
print(df_questions[['question_text', 'source_type', 'generation_method']].head(3))

🔄 Transforming data to match database enum values...
✓ Transformed source_type: ['from_documents']
✓ Transformed generation_method: ['llm_generated']

📋 Transformed data sample:
                                       question_text     source_type  \
0  When does the health insurance enrollment peri...  from_documents   
1  What are the health insurance plan options ava...  from_documents   
2  When does health insurance coverage become eff...  from_documents   

  generation_method  
0     llm_generated  
1     llm_generated  
2     llm_generated  


In [17]:
# Create backend and repository
print("=" * 70)
print("INITIALIZING FAQ REPOSITORY - Repository Pattern")
print("=" * 70)
print()

backend = BackendFactory.create_backend(config)
faq_repo = FAQRepository(backend)

print(f"✓ Backend created: {backend.__class__.__name__}")
print(f"✓ Repository created: FAQRepository")
print()

2025-11-02 14:42:13.495 | DEBUG    | database.backends.factory:create_backend:116 - Returning existing backend singleton


INITIALIZING FAQ REPOSITORY - Repository Pattern

✓ Backend created: SQLiteBackend
✓ Repository created: FAQRepository



In [18]:
# Save questions to faq_questions table using FAQRepository
print("=" * 70)
print("INGESTING QUESTIONS - Repository Pattern")
print("=" * 70)
print()

print("📊 Before ingestion:")
stats_before = faq_repo.get_faq_stats()
print(f"   Total questions: {stats_before.get('total_questions', 0)}")
print()

print("💾 Ingesting questions using FAQRepository...")
try:
    result = faq_repo.ingest_questions(
        questions_df=df_questions,
        validate=True,
        if_exists='append'
    )
    
    print(f"✅ SUCCESS")
    print(f"   Rows inserted: {result.get('rows_inserted', 0)}")
    print(f"   Message: {result.get('message', 'Ingestion completed')}")
except Exception as e:
    print(f"❌ FAILED")
    print(f"   Error: {str(e)}")
    raise

print()

# Verify
stats_after = faq_repo.get_faq_stats()
print("📊 After ingestion:")
print(f"   Total questions: {stats_after.get('total_questions', 0)}")
print()
print("=" * 70)

2025-11-02 14:42:13.509 | DEBUG    | database.repository.faq_repository:get_faq_stats:1109 - Calculating FAQ statistics
2025-11-02 14:42:13.513 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows


INGESTING QUESTIONS - Repository Pattern

📊 Before ingestion:


2025-11-02 14:42:13.515 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:13.515 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:13.517 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:13.517 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 0 rows
2025-11-02 14:42:13.517 | DEBUG    | database.repository.base:execute_query:590 - Query returned 0 rows
2025-11-02 14:42:13.519 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 0 rows
2025-11-02 14:42:13.519 | DEBUG    | database.repository.base:execute_query:590 - Query returned 0 rows
2025-11-02 14:42:13.521 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 0 rows
2025-11-02 14:42:13.521 | DEBUG    | database.repository.base:execute_query:590 - Query returned 0 rows
2025-11-02 14:42:13.522 | DEBUG 

   Total questions: 0

💾 Ingesting questions using FAQRepository...
✅ SUCCESS
   Rows inserted: 74
   Message: Successfully inserted 74 rows into faq_questions

📊 After ingestion:
   Total questions: 74



In [19]:
# Save question sources to faq_question_sources table using FAQRepository
print()
print("=" * 70)
print("INGESTING QUESTION SOURCES - Repository Pattern")
print("=" * 70)
print()

print("💾 Ingesting question sources using FAQRepository...")
try:
    # Remove 'source_id' column if present (auto-generated by database)
    if 'source_id' in df_question_sources.columns:
        df_to_ingest = df_question_sources.drop(columns=['source_id'])
    else:
        df_to_ingest = df_question_sources
    
    result = faq_repo.ingest_question_sources(
        sources_df=df_to_ingest,
        validate=True,
        validate_fk=True  # Validate that question_ids exist
    )
    
    print(f"✅ SUCCESS")
    print(f"   Rows inserted: {result.get('rows_inserted', 0)}")
    print(f"   Message: {result.get('message', 'Ingestion completed')}")
except Exception as e:
    print(f"❌ FAILED")
    print(f"   Error: {str(e)}")
    raise

print()
print("=" * 70)
print("✅ INGESTION COMPLETE")
print("=" * 70)

2025-11-02 14:42:13.569 | INFO     | database.repository.faq_repository:ingest_question_sources:1530 - Ingesting 74 question sources in bulk
2025-11-02 14:42:13.575 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 74 rows
2025-11-02 14:42:13.575 | DEBUG    | database.repository.base:execute_query:590 - Query returned 74 rows
2025-11-02 14:42:13.577 | DEBUG    | database.backends.sqlite.backend:begin_transaction:510 - Transaction started
2025-11-02 14:42:13.579 | DEBUG    | database.repository.base:transaction:366 - Transaction started
2025-11-02 14:42:13.581 | INFO     | database.backends.sqlite.backend:ingest_dataframe:426 - Inserted 74 rows into faq_question_sources
2025-11-02 14:42:13.581 | DEBUG    | database.backends.sqlite.backend:commit:529 - Transaction committed
2025-11-02 14:42:13.583 | DEBUG    | database.repository.base:transaction:371 - Transaction committed successfully
2025-11-02 14:42:13.583 | INFO     | database.repository.faq_repository


INGESTING QUESTION SOURCES - Repository Pattern

💾 Ingesting question sources using FAQRepository...
✅ SUCCESS
   Rows inserted: 74
   Message: Successfully inserted 74 rows into faq_question_sources

✅ INGESTION COMPLETE


In [20]:
# Display sample data from database using Repository
print()
print("=" * 70)
print("VERIFICATION - Repository Pattern")
print("=" * 70)
print()

print("📋 Sample questions from repository:")
# Use repository method instead of direct backend query
sample_questions = faq_repo.list_questions(limit=5, order_by="created_at DESC")
df_sample_questions = pd.DataFrame(sample_questions)
print(df_sample_questions)
print()

2025-11-02 14:42:13.593 | DEBUG    | database.repository.faq_repository:list_questions:270 - Listing questions (status=None, source_type=None, generation_method=None, limit=5, offset=None)
2025-11-02 14:42:13.595 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 5 rows
2025-11-02 14:42:13.595 | DEBUG    | database.repository.base:execute_query:590 - Query returned 5 rows
2025-11-02 14:42:13.597 | DEBUG    | database.repository.faq_repository:list_questions:311 - Found 5 questions



VERIFICATION - Repository Pattern

📋 Sample questions from repository:
                                         question_id  \
0  e23980ba9d88f7cc45875d2ebd8133987525562dedf681...   
1  ecec0d9bc8eac3f215fdb7bd94b920941defc1253ebd61...   
2  171e88489d4a5f5d2fec88f8d56c6af5a7b87ee4383ed2...   
3  c23c53660eeb5d351072fba33ab292f888bed81ae64935...   
4  18cf69d1b1df7f920c59a2bb8c17f8e790d8afdf8324d8...   

                                       question_text     source_type  \
0  Is multi-factor authentication required for ac...  from_documents   
1             How often should passwords be changed?  from_documents   
2  What are the password requirements for employees?  from_documents   
3     What class should I book for domestic flights?  from_documents   
4    Do I need manager approval for business travel?  from_documents   

  generation_method  status                created_at  \
0     llm_generated  active  2025-10-29T19:31:55.254Z   
1     llm_generated  active  2025-10-29T19:3

In [21]:
print("📋 Sample question sources from repository:")
# Use repository method instead of direct backend query
sample_sources = faq_repo.list_question_sources(limit=5, order_by="created_at DESC")
df_sample_sources = pd.DataFrame(sample_sources)
print(df_sample_sources)
print()

# Get overall stats
stats = faq_repo.get_faq_stats()
print("📊 Repository Statistics:")
print(f"   Total questions: {stats.get('total_questions', 0)}")
print(f"   Total answers: {stats.get('total_answers', 0)}")
print(f"   Questions by status: {stats.get('questions_by_status', {})}")
print(f"   Questions without answers: {stats.get('questions_without_answers', 0)}")
print()

print("=" * 70)
print("✅ VERIFICATION COMPLETE")
print("=" * 70)

2025-11-02 14:42:13.608 | DEBUG    | database.repository.faq_repository:list_question_sources:1432 - Listing question sources (question_id=None, is_valid=None, is_primary=None, limit=5)
2025-11-02 14:42:13.610 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 5 rows
2025-11-02 14:42:13.610 | DEBUG    | database.repository.base:execute_query:590 - Query returned 5 rows
2025-11-02 14:42:13.612 | DEBUG    | database.repository.faq_repository:list_question_sources:1473 - Found 5 question sources
2025-11-02 14:42:13.616 | DEBUG    | database.repository.faq_repository:get_faq_stats:1109 - Calculating FAQ statistics
2025-11-02 14:42:13.617 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:13.617 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:13.617 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:13.619 

📋 Sample question sources from repository:
   source_id                                        question_id  \
0          1  f29aefee41298f9eddc31cbbc10e3970dc067f22143c7f...   
1          2  b753cc4091103cd521842592a7ee66188af23b39a48596...   
2          3  4641cee4e692f548be4f9de631211fdd1903ea1a23060c...   
3          4  8d64db97b362e163b244cf39b6314ecb09a7525b11b0df...   
4          5  7b8d4cc8962467ca328c7cfd748d24322b4414238875f9...   

                                    content_checksum  is_primary_source  \
0  370cd156d5ae42b3bb93fe51096b05d8dc2aee7358aaee...                  1   
1  370cd156d5ae42b3bb93fe51096b05d8dc2aee7358aaee...                  1   
2  370cd156d5ae42b3bb93fe51096b05d8dc2aee7358aaee...                  1   
3  370cd156d5ae42b3bb93fe51096b05d8dc2aee7358aaee...                  1   
4  370cd156d5ae42b3bb93fe51096b05d8dc2aee7358aaee...                  1   

   contribution_weight  is_valid                valid_from valid_until  \
0                  1.0       

2025-11-02 14:42:13.623 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:13.625 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:13.625 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:13.627 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:13.627 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:13.627 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:13.629 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:13.629 | DEBUG    | database.repository.faq_repository:get_faq_stats:1193 - FAQ statistics: 74 questions, 0 answers


📊 Repository Statistics:
   Total questions: 74
   Total answers: 0
   Questions by status: {'active': 74}
   Questions without answers: 74

✅ VERIFICATION COMPLETE


In [22]:
# No need to manually close connection - repository pattern handles this
print()
print("✅ Process complete!")


✅ Process complete!
